# Melodic Pattern Recognition in Carnatic Music

In [6]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Melodic patterns (motifs and phrases), known as sañcāras, play a crucial structural and expressive role in Carnatic Music. These melodic patterns are the means through which the character of the rāga is expressed and form the basis of various improvisatory and compositional formats in the style [2,9]. There exists no definitive lists of all possible sañcāras in each rāga, rather the body of existing compositions and the living oral tradition of rāga performance act as repositories for that knowledge.

This notebook will walk through the task of identifying and annotating sañcāras from audio using the compIAM tools repository.

The methodology follows that presented in _Nuttall, Thomas, Genís Plaja-Roglans, Lara Pearson, and Xavier Sierra. "In search of Sañcāras: tradition-informed repeated melodic pattern recognition in carnatic music." (2022)._

In [7]:
from compiam import load_model
from compiam.visualisation.waveform_player import Player
from compiam.melody.pattern.sancara_search.extraction.evaluation import load_annotations_brindha, to_aeneas

## 1. Performance

We use as an example a performance of insert_raga_here by insert_performer_here, in the raga, insert_raga_here. From the saraga dataset.

In [24]:
audio_path = "/Volumes/Shruti/asplab2/cae-invar/audio/multitrack/Sharanu Janakana.mp3"
title = 'Sharanu Janakana'

In [25]:
Player(title, audio_path)

Player(value='<iframe srcdoc="&lt;!DOCTYPE html&gt;\n&lt;html lang=&quot;en&quot;&gt;\n  &lt;head&gt;\n  &lt;m…

In [ ]:
# Maybe include: Source separation?

# TODO, for each stage include a visualisation [VIS]
# [VIS] Audio waveform and playback

## 2. Feature Extraction

Two feature sets are extracted from each recording in SCV: (1) An automated transcription of the predominant sung pitch in Cents from which we derive a mask corresponding to silent/stable regions, and (2) Transformation-invariant melodic features extracted using a Complex Autoencoder (CAE) from audio in CQT representation, which we use for self-similarity computation.

### 2.1 Predominant Pitch Track

Load FTA-Net predominant pitch extraction trained on carnatic dataset

In [ ]:
ftanet = load_model('melody:ftanet-carnatic')

Extract predominant pitch track

In [ ]:
pitch_track = ftanet.predict(audio_path)
pitch = pitch_track[:,1]
time  = pitch_track[:,0]
timestep  = time[2]-time[1]

In [ ]:
# [VIS] Pitch track + waveform + audio playback

# Pitch distribution using pypeaks

Relative to other musical styles, Carnatic Music is heavily ornamented. This ornamentation is not superficial decoration but rather is integral to musical meaning [2]. The ornaments, known as gamakas, can greatly alter the sound of the notated svaras (notes); for example, some gamakas do not rest at all on the theoretical pitch of the notated svara, and instead involve oscillations between pitches either side of it [3,4]. This oscillatory movement is particularly characteristic of the style, and can often subsume individual svaras [4]. The surface effect on the melodic

#### 2.1.1 Silence and Stability

In [ ]:
# Stability Track
from compiam.utils.pitch import extract_stability_mask
import numpy as np

stability_mask = extract_stability_mask(
    pitch=pitch, 
    min_stab_sec=1.0, 
    hop_sec=0.2,
    var=60,
    timestep=timestep)

# Silence Mask
silence_mask = pitch==0

exclusion_mask = np.logical_or(silence_mask==1, stability_mask==1)

#### 2.1.2 Visualising Pitch

In [ ]:
# Pitch curve
# Load expected notes for this svara
# Can you see more peaks than svaras?

### 2.2 Melodic Feature Embeddings

In [ ]:
# Pattern Extraction for a Given Audio
from compiam import load_model

# Feature Extraction
# CAE features 
cae = load_model("melody:cae-carnatic")

ampl, phase = cae.extract_features(audio_path)

# [VIS] Feature activation

#### 2.2.1 Self Similarity Matrix

In [ ]:
# [VIS] Stability and silence annotation

# Self Similarity
from compiam.melody.pattern import self_similarity

ss = self_similarity(ampl, exclusion_mask=exclusion_mask, timestep=timestep, hop_length=cae.hop_length, sr=cae.sr)
X, orig_sparse_lookup, sparse_orig_lookup, boundaries_orig, boundaries_sparse = ss

In [ ]:
# [VIS] Self similarity matrix
# [VIS] Self similarity matrix with boundaries annotated
# [VIS] Self similarity matrix full silent and stable regions annotated

# Segment Extraction

# Segment Grouping

## Pattern Recognition